In [2]:
import gzip
import json
import csv
import re
with gzip.open(r"C:\Users\duphong.hua\Downloads\2024-06-19-16-59-08-c0e70d32-8608-4e8c-ab6a-9bb87d5d57b1.jsonl.gz", mode="rt") as f:
    data = [json.loads(line) for line in f]

In [48]:
data

[{'@severity': 'INFO',
  '@service': 'shippingfee-api',
  '@az': 'ap-vn-2-general-a',
  '@idc': '',
  '@message': "2024-06-19 15:39:22.148|INFO|normal-4896a8ce546a4dc284d352930c748e12|_calculate_hub_shippingfee done, grayscale_toggle: 1, distance: 5628.0, fee_config: {'is_round_up': True, 'unit_fee': 3750, 'pickdrop_fee': 1000, 'min_fee': 13500, 'hub_config_source': 'admin', 'discount_rate': 0.7}, group_member_fee_list: [{'shipping_fee_no_surge': 13500.0, 'fee_config': {'is_round_up': True, 'hub_config_source': 'admin', 'surge_rate': 1.0, 'min_fee': 13500, 'unit_fee': 0, 'is_decoupled': True}, 'shipping_fee': 13500, 'distance': 3100}, {'shipping_fee_no_surge': 13500.0, 'fee_config': {'is_round_up': True, 'hub_config_source': 'admin', 'surge_rate': 1.0, 'min_fee': 13500, 'unit_fee': 0, 'is_decoupled': True}, 'shipping_fee': 13500, 'distance': 2800}], final_shipping_fee: 27000, total_single_shipping_fee: 27000, hub_stack_shipping_fee: 23105.0, hub_calculate_details: {'original_pickup_poi

In [82]:
header = ['@severity',	'@service',	'@az',	'@idc',	'@message',	'@source',	'@category',	'@env',	'@timestamp',	'@cid',	'@host']
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for i in data[0:10]:
    # Create a DictWriter object
        print(i.values())
        # # Write the rows
        writer.writerows([i])

dict_values(['INFO', 'shippingfee-api', 'ap-vn-2-general-a', '', "2024-06-19 15:39:22.148|INFO|normal-4896a8ce546a4dc284d352930c748e12|_calculate_hub_shippingfee done, grayscale_toggle: 1, distance: 5628.0, fee_config: {'is_round_up': True, 'unit_fee': 3750, 'pickdrop_fee': 1000, 'min_fee': 13500, 'hub_config_source': 'admin', 'discount_rate': 0.7}, group_member_fee_list: [{'shipping_fee_no_surge': 13500.0, 'fee_config': {'is_round_up': True, 'hub_config_source': 'admin', 'surge_rate': 1.0, 'min_fee': 13500, 'unit_fee': 0, 'is_decoupled': True}, 'shipping_fee': 13500, 'distance': 3100}, {'shipping_fee_no_surge': 13500.0, 'fee_config': {'is_round_up': True, 'hub_config_source': 'admin', 'surge_rate': 1.0, 'min_fee': 13500, 'unit_fee': 0, 'is_decoupled': True}, 'shipping_fee': 13500, 'distance': 2800}], final_shipping_fee: 27000, total_single_shipping_fee: 27000, hub_stack_shipping_fee: 23105.0, hub_calculate_details: {'original_pickup_points': 2, 'dropoff_points': 2, 'lng_lat_compare_pr

In [4]:
object = []
for i in data[0:]:
    object.append(i['@message'].split(','))

In [49]:
def parse_log_entries(log_entries):
    data_dict = {}
    current_key = None
    multi_line_value = []

    for entry in log_entries:
        entry = entry.strip()
        if ': ' in entry:
            if current_key and multi_line_value:
                data_dict[current_key] = ' '.join(multi_line_value).strip()
                multi_line_value = []

            key, value = entry.split(': ', 1)
            key = key.strip()
            value = value.strip()
            
            if value.startswith('{') or value.startswith('['):
                current_key = key
                multi_line_value.append(value)
            else:
                data_dict[key] = value
                current_key = None
        elif current_key:
            multi_line_value.append(entry.strip())
    
    if current_key and multi_line_value:
        data_dict[current_key] = ' '.join(multi_line_value).strip()
    
    return data_dict

def remove_special_characters(value):
    # return re.sub(r'[^A-Za-z0-9\s]', '', value)
    # def remove_special_characters(value):
    return re.sub(r"[,{]'", '', value)

In [53]:
# cleaned_data = {key: remove_special_characters(str(object)) for key, value in parse_log_entries(object).items()}
with open('log_entries.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(remove_special_characters(parse_log_entries(object[0]).keys()))
    writer.writerow(['grayscale_toggle',	'distance',	'fee_config',	'unit_fee',	'pickdrop_fee',	'min_fee',	'hub_config_source',	'discount_rate',	'group_member_fee_list',	'fee_config',	'surge_rate',	'is_decoupled',	'shipping_fee',	'distance',	'shipping_fee_no_surge',	'final_shipping_fee',	'total_single_shipping_fee',	'hub_stack_shipping_fee',	'hub_calculate_details',	'dropoff_points',	'lng_lat_compare_precision',	'package_increase_original_fee_replace',	'unique_pickup_points_toggle',	'original_dropoff_points',	'hub_stack_fee',	'pickup_points',	'package_decrease_original_fee_replace',	'discount_fee',	'unique_dropoff_points_toggle',	'original_delivery_info',	'delivery_order_ids',	'hub_extra_fee',])
    for i in object:
    # Write the header
    # Write the row of data
        writer.writerow((parse_log_entries(i).values()))

print("CSV file created successfully.")


CSV file created successfully.


In [51]:
import pandas as pd 
df = pd.read_csv('log_entries.csv')
df['discount_rate'] = [remove_special_characters(i) for i in df['discount_rate'].to_list()]
df['hub_extra_fee'] = [remove_special_characters(str(i)) for i in df['hub_extra_fee'].to_list()]
df['delivery_order_ids'] = [remove_special_characters(i) for i in df['delivery_order_ids'].to_list()]
df['discount_rate'] = [i.replace('}','') for i in df['discount_rate'].to_list()]


In [52]:
df  
# [str(i).replace('}','') for i in df['hub_extra_fee'].to_list()]

,grayscale_toggle,distance,fee_config,unit_fee,pickdrop_fee,min_fee,hub_config_source,discount_rate,group_member_fee_list,fee_config.1,...,unique_pickup_points_toggle,original_dropoff_points,hub_stack_fee,pickup_points,package_decrease_original_fee_replace,discount_fee,unique_dropoff_points_toggle,original_delivery_info,delivery_order_ids,hub_extra_fee
0,1,5628.00,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,21105.000,2,False,18900.0,False,{'shipping_fee': 27000.0,[513351464 513368655]},2000}
1,1,2075.69,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513371459]},2000}
2,1,2219.30,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513373015]},2000}
3,1,2052.69,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513373080]},2000}
4,1,2123.69,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 27000.0,[513365916 513369305]},2000}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7511,1,4836.10,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,18135.375,2,False,18900.0,False,{'shipping_fee': 13500.0,[513368522]},2000}
7512,1,1888.70,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513365720]},2000}
7513,1,3231.80,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513367080]},2000}
7514,1,3048.39,{'is_round_up': True,0,1000,13500,'admin',0.7,[{'shipping_fee_no_surge': 13500.0,{'is_round_up': True,...,False,2,13500.000,2,False,18900.0,False,{'shipping_fee': 13500.0,[513367854]},2000}
